In [1]:
from importlib import reload
import work2.tools.preprocess as preprocess

reload(preprocess)

from work2.tools.preprocess import load_datasets, preprocess_mushrooms_datasets
import pandas as pd


In [2]:
# Load mushroom train and test dataframes

train_dfs = load_datasets('datasetsCBR/mushroom/mushroom.fold.*.train.arff')
test_dfs = load_datasets('datasetsCBR/mushroom/mushroom.fold.*.test.arff')

train_dfs[0].head()

,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,b'x',b's',b'w',b't',b'l',b'f',b'c',b'b',b'k',b'e',...,b'w',b'w',b'p',b'w',b'o',b'p',b'k',b'n',b'm',b'e'
1,b'k',b'y',b'e',b'f',b's',b'f',b'c',b'n',b'b',b't',...,b'p',b'w',b'p',b'w',b'o',b'e',b'w',b'v',b'd',b'p'
2,b'x',b'y',b'y',b't',b'a',b'f',b'c',b'b',b'n',b'e',...,b'w',b'w',b'p',b'w',b'o',b'p',b'k',b's',b'm',b'e'
3,b'x',b'y',b'w',b't',b'p',b'f',b'c',b'n',b'k',b'e',...,b'w',b'w',b'p',b'w',b'o',b'p',b'k',b'v',b'u',b'p'
4,b'x',b'y',b'n',b'f',b'f',b'f',b'c',b'n',b'b',b't',...,b'w',b'p',b'p',b'w',b'o',b'e',b'w',b'v',b'l',b'p'


In [3]:
# Check if there are any missing values in the dfs
pd.concat(test_dfs + train_dfs).isnull().sum()

cap-shape                   0
cap-surface                 0
cap-color                   0
bruises?                    0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
class                       0
dtype: int64

In [4]:
# Preprocess the mushrooms train and test dataframes
# For this dataset, we only need to convert the data to numerical values (via Label Encoding) as 
# there are no missing values or different ranges (hence no normalisation required)
for df in train_dfs + test_dfs:
    preprocess_mushrooms_datasets(df)

train_dfs[0]

,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,5,2,8,1,3,1,0,0,4,0,...,7,7,0,2,1,4,2,2,3,0
1,3,3,2,0,7,1,0,1,0,1,...,6,7,0,2,1,0,7,4,0,1
2,5,3,9,1,0,1,0,0,5,0,...,7,7,0,2,1,4,2,3,3,0
3,5,3,8,1,6,1,0,1,4,0,...,7,7,0,2,1,4,2,4,5,1
4,5,3,4,0,2,1,0,1,0,1,...,7,6,0,2,1,0,7,4,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7308,5,3,4,1,3,1,0,0,10,0,...,7,7,0,2,1,4,2,3,4,0
7309,5,3,9,0,2,1,0,0,7,0,...,0,4,0,2,1,2,1,4,1,1
7310,5,2,8,0,1,1,1,1,5,0,...,7,7,0,2,1,4,2,4,0,1
7311,2,3,4,0,2,1,0,1,0,1,...,7,7,0,2,1,0,7,4,4,1
